In [ ]:
# -*- coding: utf-8 -*-
"""
Zoptymalizowana analiza 3 kluczowych parametrów sieci neuronowej
"""

import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import time
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

class ParameterAnalyzer:
    def __init__(self, X_train, y_train, X_test, y_test, class_names):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.class_names = class_names
        self.results = []

    def test_parameter(self, param_name, param_values, base_config, n_runs=3):
        """
        Testuje wpływ konkretnego parametru na wydajność sieci
        """
        print(f"\n{'='*60}")
        print(f"TESTOWANIE PARAMETRU: {param_name}")
        print(f"{'='*60}")

        param_results = []

        for value in param_values:
            print(f"\nTestowanie {param_name} = {value}")

            # Przygotowanie konfiguracji
            config = base_config.copy()

            run_results = []

            for run in range(n_runs):
                print(f"  Uruchomienie {run + 1}/{n_runs}...", end=" ")

                start_time = time.time()

                # Tworzenie sieci z odpowiednimi parametrami
                if param_name == "learning_rate":
                    nn = OptimizedNeuralNetwork(
                        input_size=config['input_size'],
                        hidden1_size=256,
                        hidden2_size=128,
                        output_size=config['output_size'],
                        learning_rate=value,
                        lambda_reg=config['lambda_reg'],
                        dropout_rate=config['dropout_rate']
                    )
                elif param_name == "liczba_neuronow":
                    nn = self.create_network_neurons(value, config)
                elif param_name == "dropout_rate":
                    nn = OptimizedNeuralNetwork(
                        input_size=config['input_size'],
                        hidden1_size=256,
                        hidden2_size=128,
                        output_size=config['output_size'],
                        learning_rate=config['learning_rate'],
                        lambda_reg=config['lambda_reg'],
                        dropout_rate=value
                    )
                else:
                    nn = OptimizedNeuralNetwork(
                        input_size=config['input_size'],
                        hidden1_size=256,
                        hidden2_size=128,
                        output_size=config['output_size'],
                        learning_rate=config['learning_rate'],
                        lambda_reg=config['lambda_reg'],
                        dropout_rate=config['dropout_rate']
                    )

                # Trening (krótszy dla analizy)
                nn.train_batch(
                    self.X_train, self.y_train,
                    batch_size=128,
                    epochs=150,  # Więcej epok dla lepszych wyników
                    X_val=None, y_val=None
                )

                # Ewaluacja
                train_acc, _, _ = nn.evaluate(self.X_train, self.y_train)
                test_acc, _, _ = nn.evaluate(self.X_test, self.y_test)

                training_time = time.time() - start_time

                run_results.append({
                    'train_accuracy': train_acc,
                    'test_accuracy': test_acc,
                    'training_time': training_time
                })

                print(f"Train: {train_acc:.3f}, Test: {test_acc:.3f}")

            # Obliczenie statystyk dla danej wartości parametru
            avg_train = np.mean([r['train_accuracy'] for r in run_results])
            avg_test = np.mean([r['test_accuracy'] for r in run_results])
            std_train = np.std([r['train_accuracy'] for r in run_results])
            std_test = np.std([r['test_accuracy'] for r in run_results])
            max_test = max([r['test_accuracy'] for r in run_results])
            avg_time = np.mean([r['training_time'] for r in run_results])

            param_results.append({
                'parameter': param_name,
                'value': value,
                'avg_train_acc': avg_train,
                'avg_test_acc': avg_test,
                'std_train_acc': std_train,
                'std_test_acc': std_test,
                'max_test_acc': max_test,
                'avg_training_time': avg_time,
                'all_runs': run_results
            })

            print(f"  Średnie wyniki: Train={avg_train:.3f}±{std_train:.3f}, Test={avg_test:.3f}±{std_test:.3f}, Max Test={max_test:.3f}")

        self.results.extend(param_results)
        return param_results

    def create_network_neurons(self, neurons, config):
        """Tworzy sieć z różną liczbą neuronów w warstwach ukrytych"""
        return OptimizedNeuralNetwork(
            input_size=config['input_size'],
            hidden1_size=neurons,
            hidden2_size=neurons//2,
            output_size=config['output_size'],
            learning_rate=config['learning_rate'],
            lambda_reg=config['lambda_reg'],
            dropout_rate=config['dropout_rate']
        )

    def plot_parameter_comparison(self, param_results, param_name):
        """Tworzy wykresy porównawcze dla parametru"""
        values = [r['value'] for r in param_results]
        avg_train = [r['avg_train_acc'] for r in param_results]
        avg_test = [r['avg_test_acc'] for r in param_results]
        std_train = [r['std_train_acc'] for r in param_results]
        std_test = [r['std_test_acc'] for r in param_results]

        plt.figure(figsize=(12, 5))

        # Wykres dokładności
        plt.subplot(1, 2, 1)
        x_pos = np.arange(len(values))

        plt.errorbar(x_pos, avg_train, yerr=std_train, label='Train', marker='o', capsize=5)
        plt.errorbar(x_pos, avg_test, yerr=std_test, label='Test', marker='s', capsize=5)

        plt.xlabel(param_name)
        plt.ylabel('Accuracy')
        plt.title(f'Wpływ {param_name} na dokładność')
        plt.xticks(x_pos, values, rotation=45)
        plt.legend()
        plt.grid(True, alpha=0.3)

        # Wykres czasu treningu
        plt.subplot(1, 2, 2)
        times = [r['avg_training_time'] for r in param_results]
        plt.bar(x_pos, times, alpha=0.7)
        plt.xlabel(param_name)
        plt.ylabel('Czas treningu (s)')
        plt.title(f'Wpływ {param_name} na czas treningu')
        plt.xticks(x_pos, values, rotation=45)
        plt.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()

    def generate_report(self):
        """Generuje raport z wynikami analizy"""
        print("\n" + "="*80)
        print("RAPORT Z ANALIZY 3 KLUCZOWYCH PARAMETRÓW")
        print("="*80)

        # Grupowanie wyników według parametrów
        params = {}
        for result in self.results:
            param_name = result['parameter']
            if param_name not in params:
                params[param_name] = []
            params[param_name].append(result)

        for param_name, param_results in params.items():
            print(f"\n{param_name.upper()}:")
            print("-" * 50)

            best_result = max(param_results, key=lambda x: x['avg_test_acc'])

            for result in param_results:
                status = " ← NAJLEPSZY" if result == best_result else ""
                print(f"  {result['value']}: "
                      f"Train={result['avg_train_acc']:.3f}±{result['std_train_acc']:.3f}, "
                      f"Test={result['avg_test_acc']:.3f}±{result['std_test_acc']:.3f}, "
                      f"Max={result['max_test_acc']:.3f}{status}")

            self.plot_parameter_comparison(param_results, param_name)

# GŁÓWNA CZĘŚĆ ANALIZY
print("Rozpoczynanie analizy 3 kluczowych parametrów sieci neuronowej...")

# Konfiguracja bazowa (zakładając, że masz już zdefiniowane zmienne)
base_config = {
    'input_size': input_size,
    'output_size': output_size,
    'learning_rate': 0.01,
    'lambda_reg': 0.001,
    'dropout_rate': 0.2
}

# Utworzenie analizatora
analyzer = ParameterAnalyzer(X_train, y_train, X_test, y_test, class_names)

# 1. ANALIZA WSPÓŁCZYNNIKA UCZENIA (LEARNING RATE)
print("1. Testowanie wpływu współczynnika uczenia...")
lr_results = analyzer.test_parameter(
    param_name="learning_rate",
    param_values=[0.001, 0.01, 0.05, 0.1],  # 4 różne wartości
    base_config=base_config,
    n_runs=3
)

# 2. ANALIZA LICZBY NEURONÓW W WARSTWACH UKRYTYCH
print("\n2. Testowanie wpływu liczby neuronów...")
neurons_results = analyzer.test_parameter(
    param_name="liczba_neuronow",
    param_values=[64, 128, 256, 512],  # 4 różne wartości
    base_config=base_config,
    n_runs=3
)

# 3. ANALIZA DROPOUT RATE
print("\n3. Testowanie wpływu dropout rate...")
dropout_results = analyzer.test_parameter(
    param_name="dropout_rate",
    param_values=[0.0, 0.2, 0.4, 0.6],  # 4 różne wartości
    base_config=base_config,
    n_runs=3
)

# GENEROWANIE RAPORTU
analyzer.generate_report()

# PODSUMOWANIE NAJLEPSZYCH KONFIGURACJI
print("\n" + "="*80)
print("PODSUMOWANIE NAJLEPSZYCH KONFIGURACJI")
print("="*80)

# Znajdź najlepsze wartości dla każdego parametru
best_configs = {}
for result in analyzer.results:
    param = result['parameter']
    if param not in best_configs or result['avg_test_acc'] > best_configs[param]['avg_test_acc']:
        best_configs[param] = result

print("Najlepsze wartości dla 3 analizowanych parametrów:")
for param, config in best_configs.items():
    print(f"  {param}: {config['value']} (Test Acc: {config['avg_test_acc']:.3f})")

# TRENOWANIE SIECI Z NAJLEPSZYMI PARAMETRAMI
print(f"\n{'='*60}")
print("TRENOWANIE SIECI Z NAJLEPSZYMI PARAMETRAMI")
print("="*60)

# Utworzenie sieci z najlepszymi parametrami
best_lr = best_configs.get('learning_rate', {}).get('value', 0.01)
best_dropout = best_configs.get('dropout_rate', {}).get('value', 0.2)
best_neurons = best_configs.get('liczba_neuronow', {}).get('value', 256)

print(f"Konfiguracja najlepszej sieci:")
print(f"  Learning rate: {best_lr}")
print(f"  Dropout rate: {best_dropout}")
print(f"  Liczba neuronów: {best_neurons}")
print(f"  Regularyzacja: {base_config['lambda_reg']} (stała)")

final_nn = OptimizedNeuralNetwork(
    input_size=input_size,
    hidden1_size=best_neurons,
    hidden2_size=best_neurons//2,
    output_size=output_size,
    learning_rate=best_lr,
    lambda_reg=base_config['lambda_reg'],
    dropout_rate=best_dropout
)

print("\nTrenowanie finalnej sieci z najlepszymi parametrami...")
final_train_losses, final_val_losses = final_nn.train_batch(
    X_train, y_train,
    X_val=X_test, y_val=y_test,
    batch_size=128,
    epochs=200
)

# Finalna ewaluacja
final_accuracy, y_true_final, y_pred_final = final_nn.evaluate(X_test, y_test)
print(f"\nFinalna dokładność na zbiorze testowym: {final_accuracy:.4f}")

print("\nFinalny classification report:")
print(classification_report(y_true_final, y_pred_final, target_names=class_names))

# TABELA WYNIKÓW DO SPRAWOZDANIA
print(f"\n{'='*80}")
print("TABELA WYNIKÓW DO SPRAWOZDANIA")
print("="*80)

results_table = pd.DataFrame(analyzer.results)
print("\nWyniki analizy 3 parametrów:")
summary_table = results_table[['parameter', 'value', 'avg_train_acc', 'avg_test_acc', 'std_test_acc', 'max_test_acc']].round(4)
print(summary_table)

# Zapisanie wyników do CSV
results_table.to_csv('wyniki_analizy_3_parametrow.csv', index=False)
print("\nWyniki zapisane do pliku: wyniki_analizy_3_parametrow.csv")

# PODSUMOWANIE KOŃCOWE
print(f"\n{'='*80}")
print("ANALIZA 3 PARAMETRÓW ZAKOŃCZONA")
print("="*80)
print("Przeanalizowane parametry:")
print("1. Learning Rate (współczynnik uczenia)")
print("2. Liczba neuronów w warstwach ukrytych")
print("3. Dropout Rate (współczynnik dropout)")
print("\nTe 3 parametry mają największy wpływ na wydajność sieci neuronowej.")
print("Wyniki są gotowe do wykorzystania w sprawozdaniu!")
print("="*80)

Rozpoczynanie analizy 3 kluczowych parametrów sieci neuronowej...
1. Testowanie wpływu współczynnika uczenia...

TESTOWANIE PARAMETRU: learning_rate

Testowanie learning_rate = 0.001
  Uruchomienie 1/3... Epoch 0, Train loss: 1.8671, Train acc: 0.3391
